In [6]:
import cx_Oracle
import os

In [7]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [4]:
cur = connection.cursor()

NameError: name 'connection' is not defined

In [5]:
cur.execute('DELETE FROM "C##PPAWLUK"."TEMAT_DOKUMENTU"')
cur.execute('commit')

NameError: name 'cur' is not defined

In [5]:
dokumentsName = cur.execute('select distinct nazwa_dokumentu from dokument')

In [6]:
names = []

In [7]:
for dokument in dokumentsName:
    names.append(dokument[0])

In [8]:
dokumentsText = []

In [9]:
j = 0
for name in names:
    dokumentsText.append({'name': name, 'text': ''})
    text = cur.execute('select tekst from dokument where nazwa_dokumentu = \'{}\''.format(name))
    for t in text:
        dokumentsText[j]['text'] = dokumentsText[j]['text'] + t[0]
    j = j + 1

In [10]:
allTekst = ''

In [11]:
for z in dokumentsText:
    allTekst = allTekst + z['text']

In [12]:
allTekst = allTekst.split(' ')
allTekst.remove('')
allTekst = list(set(allTekst))

In [13]:
matrix = []

In [14]:
for word in allTekst:
    for dokument in names:
        matrix.append({"name":dokument,"word": word,'frequency':0})

In [15]:
for dokument in dokumentsText:
    text = dokument['text']
    text = text.split(' ')
    text.remove('')
    for m in matrix:
        if(m['name'] == dokument['name']):
            m['frequency'] = text.count(m['word'])

In [16]:
def themaDoc(matrix,nameDoc):
    tmpMatrix = []
    sum = 0
    for m in matrix:
        if(m['name'] == nameDoc):
            sum = sum + m['frequency']
            if(m['frequency'] > 1):
                tmpMatrix.append(m)
    sortMatrix = sorted(tmpMatrix, key=lambda x: x['frequency'],reverse=True)
    size = 10
    if( len(sortMatrix) < size):
        size = len(sortMatrix)
    tematy = ''
    for i in range(0,size,1):
        tematy = tematy + ' ' + sortMatrix[i]['word'] + ' ' + '{:.2f}%'.format((float(sortMatrix[i]['frequency'])/float(sum))*100)+','
        print("{} {:.2f}%".format(sortMatrix[i] ,(float(sortMatrix[i]['frequency'])/float(sum))*100))
    cur.execute('INSERT INTO "C##PPAWLUK"."TEMAT_DOKUMENTU" (NAZWA_DOKUMENTU, TEMATY) VALUES( \'{}\', \'{}\')'.format(nameDoc,tematy))
    return sortMatrix

In [17]:
tmpM = []
for name in names:
    tmpM.append(themaDoc(matrix,name))

{'name': 'D20181943.txt', 'word': 'akt', 'frequency': 22} 3.77%
{'name': 'D20181943.txt', 'word': 'prawny', 'frequency': 22} 3.77%
{'name': 'D20181943.txt', 'word': 'chorągiew', 'frequency': 17} 2.92%
{'name': 'D20181943.txt', 'word': 'w', 'frequency': 15} 2.57%
{'name': 'D20181943.txt', 'word': 'addendum', 'frequency': 15} 2.57%
{'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 14} 2.40%
{'name': 'D20181943.txt', 'word': 'miesięcy', 'frequency': 13} 2.23%
{'name': 'D20181943.txt', 'word': 'dwanaście', 'frequency': 13} 2.23%
{'name': 'D20181943.txt', 'word': 'powód', 'frequency': 12} 2.06%
{'name': 'D20181943.txt', 'word': 'barwa', 'frequency': 11} 1.89%
{'name': 'D20181937.txt', 'word': 'akt', 'frequency': 9} 4.04%
{'name': 'D20181937.txt', 'word': 'miesięcy', 'frequency': 9} 4.04%
{'name': 'D20181937.txt', 'word': 'doba', 'frequency': 9} 4.04%
{'name': 'D20181937.txt', 'word': 'dwanaście', 'frequency': 9} 4.04%
{'name': 'D20181937.txt', 'word': 'prawny', 'frequency': 9} 4.0

In [18]:
for i in tmpM:
    print(i)

[{'name': 'D20181943.txt', 'word': 'akt', 'frequency': 22}, {'name': 'D20181943.txt', 'word': 'prawny', 'frequency': 22}, {'name': 'D20181943.txt', 'word': 'chorągiew', 'frequency': 17}, {'name': 'D20181943.txt', 'word': 'w', 'frequency': 15}, {'name': 'D20181943.txt', 'word': 'addendum', 'frequency': 15}, {'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 14}, {'name': 'D20181943.txt', 'word': 'miesięcy', 'frequency': 13}, {'name': 'D20181943.txt', 'word': 'dwanaście', 'frequency': 13}, {'name': 'D20181943.txt', 'word': 'powód', 'frequency': 12}, {'name': 'D20181943.txt', 'word': 'barwa', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'plaster', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'casus', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'rozporządzenie', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'numer', 'frequency': 9}, {'name': 'D20181943.txt', 'word': 'doba', 'frequency': 9}, {'name': 'D20181943.txt', 'word': 'członek', 'frequency': 8}, {

In [19]:
cur.execute('commit')

In [20]:
cur.close()
connection.close()